In [1]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# <font color='orange'>01. load data

In [2]:
data_rootpath = r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\dev_data'

In [3]:
os.listdir(data_rootpath)

['model_dev_cb_df.csv',
 'model_dev_inner_df.csv',
 'model_valid_cb_df.csv',
 'model_valid_inner_df.csv']

In [4]:
dev_inner_df = pd.read_csv(os.path.join(data_rootpath, 'model_dev_inner_df.csv'))
valid_inner_df = pd.read_csv(os.path.join(data_rootpath, 'model_valid_inner_df.csv'))

In [5]:
# dictionary에 담기
df_dict = dict()

df_dict['dev_inner'] = dev_inner_df
df_dict['valid_inner'] = valid_inner_df

In [6]:
for key, df in df_dict.items():
    print(key, df.shape)

dev_inner (240016, 67)
valid_inner (40942, 67)


In [7]:
df_dict['dev_inner'].head(2)

,no,신청월,직군그룹,근속기간1,웰컴_소득구간,심사원장_소득구간,EXECUTE_FG,DESTRUCT_FG,Y_2017_FG,Y_2018_FG,...,BAD_금융_카드_12,BAD_금융_캐피탈_12,BAD_금융_저축은행_12,BAD_대부_12,BAD_금융_6,BAD_금융_카드_6,BAD_금융_캐피탈_6,BAD_금융_저축은행_6,BAD_대부_6,BAD
0,1,201612,NaN,5.0,5.0,0.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,201612,D,5.0,2.0,0.0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# 모형검증데이터 월별 나누기
df_dict['valid_inner_1908'] = valid_inner_df[valid_inner_df['신청월']==201908]
df_dict['valid_inner_1909'] = valid_inner_df[valid_inner_df['신청월']==201909]
df_dict['valid_inner_1910'] = valid_inner_df[valid_inner_df['신청월']==201910]

## <font color='orange'>01소득구간별 + UDIR 현황

* 주택미보유자 대상으로 해야함

In [9]:
def _pivot_grade(data, grade_var, bad_var, column_var):
    '''
    grade_var = index로 갈 변수
    bad_var = 계산할 변수 
    column_var = column으로 갈 변수
    '''
    piv = data.pivot_table(index = grade_var, columns = column_var, values = [bad_var],
                          aggfunc = ['count','sum'],
                           fill_value = 0,
                          margins = True,
                          margins_name = 'ALL'
                          )
    # 컬럼 정렬
#     piv.sort_index( axis = 1, inplace = True, level = column_var)
    # 컬럼명 변경
    piv.columns = ['TOTAL_CNT_' + column_var + '_' + str(col3) if col1 == 'count' else 'BAD_CNT_' + column_var + '_' + str(col3) for (col1, col2, col3) in piv.columns.tolist()]
    # reset index
    piv.reset_index(inplace = True, )
    
    return piv

In [10]:
seg_udir_dict = dict()
for key, df in df_dict.items():
    df = df[df['TOTAL_PRE_BAD_FG'] == 0]
    this = df[df['TOTAL_HOUSE_FG'] == 0]
    seg_udir_dict[key] = _pivot_grade(this, 'INCOME_INTERVAL','BAD','UDIR_INTERVAL')
    

In [11]:
seg_udir_dict['dev_inner']

,INCOME_INTERVAL,TOTAL_CNT_UDIR_INTERVAL_1,TOTAL_CNT_UDIR_INTERVAL_2,TOTAL_CNT_UDIR_INTERVAL_3,TOTAL_CNT_UDIR_INTERVAL_4,TOTAL_CNT_UDIR_INTERVAL_5,TOTAL_CNT_UDIR_INTERVAL_6,TOTAL_CNT_UDIR_INTERVAL_7,TOTAL_CNT_UDIR_INTERVAL_ALL,BAD_CNT_UDIR_INTERVAL_1,BAD_CNT_UDIR_INTERVAL_2,BAD_CNT_UDIR_INTERVAL_3,BAD_CNT_UDIR_INTERVAL_4,BAD_CNT_UDIR_INTERVAL_5,BAD_CNT_UDIR_INTERVAL_6,BAD_CNT_UDIR_INTERVAL_7,BAD_CNT_UDIR_INTERVAL_ALL
0,1,2082,1446,1150,1071,967,762,1924,9402,637,549,488,416,422,355,916,3783
1,2,3421,2141,2058,1677,1174,663,1070,12204,1079,884,876,689,532,288,537,4885
2,3,5979,4248,3593,2412,1390,678,730,19030,2105,1611,1448,1038,653,303,452,7610
3,4,16106,6933,4815,2912,1385,629,616,33396,5662,2987,2103,1344,674,347,335,13452
4,5,9830,6161,4263,2166,1079,441,426,24366,3608,2462,1858,990,573,235,232,9958
5,6,7782,3980,2606,1312,547,242,259,16728,2820,1732,1238,596,281,134,159,6960
6,7,5766,4643,2808,1269,542,227,257,15512,2256,1952,1287,603,263,133,139,6633
7,8,6082,3896,1915,786,336,102,135,13252,2500,1758,921,388,167,60,64,5858
8,9,8508,6230,2847,1165,457,247,281,19735,3230,2704,1358,576,251,118,117,8354
9,10,2898,2452,1232,617,334,149,182,7864,1083,986,542,281,194,77,81,3244


In [12]:
save_path = r'C:\Users\w10\Desktop\신용평가모형 세미나\2주차\데이터\SEG상세현황'

# 한 엑셀파일에 여러 sheet로 저장


save_path = os.path.join(save_path, 'SEG상세현황_결과.xlsx')
with pd.ExcelWriter(save_path) as writer:
    for key, result in seg_udir_dict.items():
        result.to_excel(writer, sheet_name = key, )